In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
dfs = pd.read_excel('2022_GPS.xlsx', sheet_name=None)

# Access individual sheets
df1 = dfs['GROUP5 CG5-1']
df2 = dfs['GROUP5 CG5-2']
df3 = dfs['GROUP1 CG5-2']

In [3]:
df3

,Base,Time (h),obs (mGal),Northing (m),Easting (m),Elevation (m),Corrected
0,base,0.000000,3439.016,4483141.352,344090.460,2032.563,NaN
1,base,1.200000,3439.036,4483141.352,344090.460,2032.563,NaN
2,NaN,1.350000,3434.815,4483190.106,343971.653,2055.956,NaN
3,NaN,1.466667,3434.812,4483188.456,343970.597,2056.168,NaN
4,NaN,1.533333,3434.837,4483187.305,343972.140,2056.196,NaN
5,NaN,1.633333,3434.848,4483186.008,343973.719,2056.129,NaN
6,NaN,1.750000,3434.860,4483191.772,343972.819,2055.757,NaN
7,NaN,1.800000,3434.868,4483187.727,343974.846,2055.898,NaN
8,NaN,1.916667,3434.884,4483186.526,343976.340,2055.811,NaN
9,NaN,2.016667,3434.907,4483185.276,343977.848,2055.762,NaN


In [4]:
def drift_correction(df):
    base_idx = np.where(df['Base'] == 'base')[0]
    
    times = df['Time (h)']
    obs = df['obs (mGal)']
    
    g_drift = np.interp(times, times[base_idx], obs[base_idx])

    drift_corrected = df["obs (mGal)"] - g_drift
    df["drift corrected"] = drift_corrected

In [5]:
def freeair_correction(df):
    elevations = df["Elevation (m)"]
    g_fa = -0.3086 * elevations

    og = df["drift corrected"]
    df["free air corrected"] = og - g_fa

In [6]:
def bouguer(df):
    elevations = df["Elevation (m)"]
    g_b = 0.0921995 * elevations

    og = df["free air corrected"]
    df["bouguer corrected"] = og - g_b

In [17]:
df = df1
drift_correction(df)
freeair_correction(df)
bouguer(df)

df_final = pd.DataFrame(index=range(24))
df_final["Group5_CG51"] = df["bouguer corrected"] 

In [18]:
df = df2
drift_correction(df)
freeair_correction(df)
bouguer(df)

df_final["Group5_CG52"] = df["bouguer corrected"] 

In [21]:
df = df3
drift_correction(df)
freeair_correction(df)
bouguer(df)

df_final["Group1_CG52"] = df["bouguer corrected"] 

,Group5_CG51,Group5_CG52,Group1_CG52
0,440.692261,440.692261,439.847649
1,440.795458,440.839955,439.847649
2,440.813563,440.823002,440.689568
3,440.816481,440.824390,440.732960
4,440.792009,440.692261,440.764313
5,440.802490,440.729823,440.761255
6,440.834696,440.774561,440.693269
7,440.788953,440.769774,440.732002
8,440.838382,440.774975,440.729690
9,440.852611,440.762907,440.742528


In [20]:
df_final.to_csv('Grav_corrected.csv', index=False)